In [ ]:
import json
import pickle

import pandas as pd
from vllm import LLM, SamplingParams

In [ ]:
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

In [4]:
def get_prompt(text):
    prompt = f"""
        <s>
        [INST] Ты умный помощник, разбирающийся в NER и Entity-level sentiment analysis в финансовых документах. Твоя задача проанализировать предоставленный текст новости, выявить в нём все компании и дать каждой компании оценку в диапазоне от 1 до 5, основываясь на предоставленном тексте, и выдать ответ в формате массива JSON объектов. Отвечай всегда в этом формате, не дописывай ничего лишнего. Пиши всегда на русском.

        Чтобы выбрать правильную оценку, следуй следующим правилам:
        1: что-то очень негативное относительно компании или дана рекомендация "продавать";
        2: что-то скорее негативное, например, вышла отчетность ниже ожиданий, проблемы на каком-нибудь заводе, санкции и пр;
        3: Нейтральная новость;
        4: Что-то положительное, например, вышла отчетность выше ожиданий, успехи на каком-нибудь заводе, новый контракт;
        5: Что-то очень положительное или есть рекомендация "покупать" или "входит в подборку наших супер-акций".

        Например, для следующего текста:

        "Бесплатных долларов больше нет Акции и инвестиции USDRUB +1.9% После февральских скачков, доллар можно было приобрести практически бесплатно, всего за 50-55 рублей. Товары в РФ на тот момент не упали в цене(и даже выросли), а значит курс валюты был занижен искусственно. Сложно сказать где остановится рост, вполне вероятно мы ещё увидим курс ниже текущего уровня. Рост импорта и выход недружественных нерезидентов с рынка, будут продолжать оказывать поддержку доллару Уже появились первые словесные интервенции Рубль перейдет к укреплению вслед за ростом цен на энергоносители — Силуанов IMOEX -0.1% Без роста курса доллара мы не увидели бы рост индекса от сентябрьским минимумов доллар прибавил более 50%, а индекс прибавил более 40% НЛМК -1.5% Пока не понятно насколько эта информация достоверна Телеграм канал BRIEF сообщил, что санкции против Лисина и НЛМК могут быть включены в 11 пакет санкций Евросоюза. После чего вероятно попадание в SDN-лист. Лично я надеюсь, что это обыкновенный вброс OZON -3.3% Акции падают после публикации хорошей отчётности. Ранее котировки неплохо выросли на ожиданиях отчётности, сегодня идёт фиксация на факте публикации  FIVE -0.2% \X5 Group продолжает изучать возможность смены юрисдикции головной компании из Нидерландов. В данный момент компания не видит возможности для переезда Сейчас все адр-гдр в трудном положении, непонятно как они будут решать эту ситуацию Татнефть ап +1.3% Компания продолжает уверенно идти к полной переработке добываемой нефти Татнефть может купить 50% турецкой топливной компании Aytemiz Akaryakit за $160 млн Сегежа +2.2% «Сегежа групп» планирует разместить в Иркутской области завод по производству CLT-панелей Whoosh +5.5% Компания сегодня выступает на конференции съезд «На СИМ», похоже инвесторам понравилось их выступление. Компания анонсировала новую модель самокатов с интерактивной панелью и ответил на опасения запрета самокатов «как в Париже» Лукойл +0.8% Инвест. дома играют в игру «кто даст больше дивиденд», пока консенсус в районе 500 рублей"

        ты должен вернуть:
        [/INST]
        [
            {{
                "company": "Whoosh WUSH",
                "score": 4.0
            }},
            {{
                "company": "X5 групп",
                "score": 3.0
            }},
            {{
                "company": "OZON",
                "score": 4.0
            }},
            {{
                "company": "НЛМК",
                "score": 2.0
            }}
        ]
        </s>
        [INST]
        Текст новости:

        "{text}"

        Ты должен вернуть:
        [/INST]
        """
    return prompt

In [5]:
messages: pd.DataFrame = pd.read_pickle("../data/sentiment_texts.pkl")
messages = messages.drop_duplicates(subset=["MessageID", "ChannelID"])
messages = messages[["MessageText", "SentimentScore"]]
messages.head()

,MessageText,SentimentScore
0,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,2
1,Ozon продолжает развивать специализированные ф...,4
2,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,4
4,​​Windfall Tax — налог на сверхприбыль. Какие ...,2
8,"У нас было 2 пакета префов Сургутнефтегаза, 75...",2


In [9]:
prompts = [get_prompt(messages.iloc[i].values[0]) for i in range(messages.shape[0])]

In [10]:
sampling_params = SamplingParams(temperature=0.6, top_p=1, max_tokens=500)
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs[:10]:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt}")
    print(f"Generated text: {generated_text}")

Processed prompts: 100%|██████████| 7256/7256 [20:21<00:00,  5.94it/s]

Prompt: 
<s>
[INST] Ты умный помощник, разбирающийся в NER и Entity-level sentiment analysis в финансовых документах. Твоя задача проанализировать предоставленный текст новости, выявить в нём все компании и дать каждой компании оценку в диапазоне от 1 до 5, основываясь на предоставленном тексте, и выдать ответ в формате массива JSON объектов. Отвечай всегда в этом формате, не дописывай ничего лишнего. Пиши всегда на русском.

Чтобы выбрать правильную оценку, следуй следующим правилам:
1: что-то очень негативное относительно компании или дана рекомендация "продавать";
2: что-то скорее негативное, например, вышла отчетность ниже ожиданий, проблемы на каком-нибудь заводе, санкции и пр;
3: Нейтральная новость;
4: Что-то положительное, например, вышла отчетность выше ожиданий, успехи на каком-нибудь заводе, новый контракт;
5: Что-то очень положительное или есть рекомендация "покупать" или "входит в подборку наших супер-акций".

Например, для следующего текста:

"Бесплатных долларов больше н

In [30]:
cleared = []
errs = 0
for i in range(len(outputs)):
    try:
        res = outputs[i].outputs[0].text
        a = res[res.find("[") : res.rfind("]") + 1].strip()
        a = json.loads(a)
        cleared.append({"corpus": messages.iloc[i].MessageText, "entities": a})
    except Exception as e:
        errs += 1

list indices must be integers or slices, not str
[{'company': 'Селигдар', 'score': 1.0}]


In [28]:
errs

7256

In [22]:
cleared[5]

{'corpus': 'Итоги недели 27.03.23 - 01.04.23  Закончился первый квартал 2023 года. Ждем скоро вал отчетов сначала по 🇺🇲США, затем по 🇷🇺РФ.   🖋 Аналитика  Выпустили на сайте обзоры по крепкому газовику Новатэк MOEX: NVTK, Globaltrans MOEX: GLTR, который собрал много денег на высоких ставках на вагоны и по трем публичным девелоперам: Etalon MOEX:ETLN, ЛСР MOEX:LSRG,  и Самолет MOEX: SMLT  Смотрели отчет у Сегежи MOEX: SGZH и сделали конспект их очень подробного звонка. Все же в холдинге АФК Система работа с акционерами одна из лучших на нашем рынке. Пост скоро на сайте.  Смотрели отчет ЦИАН MOEX: CIAN, который начал пировать на рекламных бюджетах девелоперов и риэлторов. Да и в этом году может продолжить получать большие деньги, ведь продавать стало труднее. В Лента MOEX: LNTA - посмотрели на мир стагнирующих гипермаркетов, а в «Россети Ленэнерго» MOEX: LSNG, LSNGP отчетность РСБУ успокоила любителей дивидендов.  ⚡ Важные события  🪢 Китайский гигант Aliababa NYSE: BABA хочет целых 6 IPO 

In [24]:
with open("labeled_data.pickle", "wb") as handle:
    pickle.dump(cleared, handle, protocol=pickle.HIGHEST_PROTOCOL)